# Word2Vec 개선
앞선 모델의 문제로,
1. W_in 의 비효율적인 계산
2. W_out 과 Softmax 의 비효율적인 계산
<br><br>
- 1 의 문제는 Embedding layer 를 추가함으로써 개선할 수 있다.
- 2 의 문제는 W_oout 과 Softmax 의 계산문제를 Binary Classification 문제로 바꾸어 개선한다. 동시에 Negative Sampling 을 시행한다.

In [2]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
        return None

In [3]:
# W_out 과 dot product 
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    def forward(self, h, idx):
        target_W = self.embed.farward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0],1)
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### Improved CBOW modeling

In [5]:
import sys
sys.path.append('./deep-learning-from-scratch-2/')
import numpy as np

from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss

In [10]:
class CBOW:
    def __initt__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # Initialize Weight parameter
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # Generate Layers
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sampel_size=5)
        
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        self.word_vecs = W_in
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.inlayers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

In [12]:
import sys
sys.path.append('./deep-learning-from-scratch-2/')
sys.path.append('./deep-learning-from-scratch-2/ch04/')
import numpy as np
from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from skip_gram import SkipGram
from cbow import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 1[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 2[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 3[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 4[s] | 손실 4.04
| 에폭 1 |  반복 101 / 9295 | 시간 5[s] | 손실 3.93
| 에폭 1 |  반복 121 / 9295 | 시간 6[s] | 손실 3.77
| 에폭 1 |  반복 141 / 9295 | 시간 8[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 9[s] | 손실 3.48
| 에폭 1 |  반복 181 / 9295 | 시간 10[s] | 손실 3.36
| 에폭 1 |  반복 201 / 9295 | 시간 11[s] | 손실 3.24
| 에폭 1 |  반복 221 / 9295 | 시간 12[s] | 손실 3.17
| 에폭 1 |  반복 241 / 9295 | 시간 13[s] | 손실 3.09
| 에폭 1 |  반복 261 / 9295 | 시간 14[s] | 손실 3.02
| 에폭 1 |  반복 281 / 9295 | 시간 15[s] | 손실 2.96
| 에폭 1 |  반복 301 / 9295 | 시간 17[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 18[s] | 손실 2.87
| 에폭 1 |  반복 341 / 9295 | 시간 19[s] | 손실 2.83
| 에폭 1 |  반복 361 / 9295 | 시간 20[s] | 손실 2.83
| 에폭 1 |  반복 381 / 9295 | 시간 21[s] | 손실 2.79
| 에폭 1 |  반복 401 / 9295 | 시간 22[s] | 손실 2.77
| 에폭 1 |  반복 421 / 9295 | 시간 24[s] | 손실 2.74
| 에폭 1 |  반복 441 / 9295 |

/Users/emjayahn/.pyenv/versions/3.6.0/envs/deeplearning/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
